In [1]:
import numpy as np
import pandas as pd


In [3]:
train_b = pd.read_csv("../data/train_symp_path.csv")
test_b = pd.read_csv("../data/test_symp_path.csv")
def process(df):
    df['Symptoms'] = df['Symptoms'].str.lower()  # Convert to lower case
    df['Symptoms'] = df['Symptoms'].str.replace('[^\w\s]', '')  # Remove punctuation
    df['Symptoms'] = df['Symptoms'].str.replace('\n', ' ')  # Remove punctuation
    return df
train_label = process(train_b)['Disease']
test_label = process(test_b)['Disease']

In [77]:
text_train = pd.read_csv("../data/X_train_probabilities.csv")
text_test = pd.read_csv("../data/X_test_probabilities.csv")

bert_train = pd.read_csv("../data/train_probabilities.csv")
bert_test = pd.read_csv("../data/test_probabilities.csv")

p_train = pd.read_csv("../data/train_combined_p.csv",header=None)
p_test = pd.read_csv("../data/test_combined_p.csv",header=None)

In [25]:
stacked_probabilities_train = np.hstack((p_train, text_train))
stacked_probabilities_test = np.hstack((p_test, text_test))

In [27]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
y_encoded_train = label_encoder.fit_transform(train_label)
y_encoded_test = label_encoder.fit_transform(test_label)

In [28]:
from sklearn.linear_model import LogisticRegression

fusion_model = LogisticRegression().fit(stacked_probabilities_train, y_encoded_train)

In [30]:
from sklearn.metrics import classification_report, accuracy_score

y_pred_log = fusion_model.predict(stacked_probabilities_test)

accuracy = accuracy_score(y_encoded_test, y_pred_log)
print(f'Accuracy of the fusion model: {accuracy * 100:.2f}%')


Accuracy of the fusion model: 87.80%


In [32]:
print(classification_report(y_encoded_test, y_pred_log, target_names=label_encoder.classes_))


                                                      precision    recall  f1-score   support

                                   Atopic Dermatitis       0.84      1.00      0.91        32
          Lupus and other Connective Tissue diseases       0.94      0.94      0.94        17
Scabies Lyme Diease and other Infestations and Bites       0.90      0.72      0.80        25
        Seborrheic Keratoses and other Benign Tumors       0.86      0.75      0.80         8

                                            accuracy                           0.88        82
                                           macro avg       0.89      0.85      0.86        82
                                        weighted avg       0.88      0.88      0.87        82



In [53]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.regularizers import l2


In [34]:
# Number of classes
num_classes = 4  # Assuming y_train is one-hot encoded

fusion_model = Sequential([
    Dense(64, input_shape=(stacked_probabilities_train.shape[1],), activation='relu'),
    Dropout(0.5),
    Dense(num_classes, activation='softmax')
])

fusion_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [35]:
from tensorflow.keras.utils import to_categorical

y_train_encoded = to_categorical(y_encoded_train, num_classes=4)
y_val_encoded = to_categorical(y_encoded_test, num_classes=4)

In [36]:
fusion_model.fit(stacked_probabilities_train, y_train_encoded, validation_data=(stacked_probabilities_test, y_val_encoded), epochs=10, batch_size=32)


Epoch 1/10
11/11 [==============================] - 1s 28ms/step - loss: 1.2329 - accuracy: 0.5921 - val_loss: 1.1834 - val_accuracy: 0.7561
Epoch 2/10
11/11 [==============================] - 0s 5ms/step - loss: 1.1229 - accuracy: 0.7039 - val_loss: 1.1089 - val_accuracy: 0.7927
Epoch 3/10
11/11 [==============================] - 0s 5ms/step - loss: 1.0401 - accuracy: 0.7855 - val_loss: 1.0383 - val_accuracy: 0.7683
Epoch 4/10
11/11 [==============================] - 0s 5ms/step - loss: 0.9621 - accuracy: 0.7976 - val_loss: 0.9714 - val_accuracy: 0.7805
Epoch 5/10
11/11 [==============================] - 0s 5ms/step - loss: 0.8799 - accuracy: 0.8550 - val_loss: 0.9092 - val_accuracy: 0.8049
Epoch 6/10
11/11 [==============================] - 0s 5ms/step - loss: 0.7964 - accuracy: 0.8580 - val_loss: 0.8515 - val_accuracy: 0.8049
Epoch 7/10
11/11 [==============================] - 0s 6ms/step - loss: 0.7238 - accuracy: 0.8731 - val_loss: 0.7973 - val_accuracy: 0.7927
Epoch 8/10
11/11 [=

In [37]:
val_loss, val_accuracy = fusion_model.evaluate(stacked_probabilities_test, y_val_encoded)
print(f"Validation Accuracy: {val_accuracy:.4f}")


3/3 [==============================] - 0s 3ms/step - loss: 0.6663 - accuracy: 0.8049
Validation Accuracy: 0.8049


In [39]:
num_classes = 4  # Assuming y_train is one-hot encoded

fusion_model = Sequential([
    Dense(64, input_shape=(stacked_probabilities_train.shape[1],), activation='relu'),
    Dropout(0.5),
    Dense(32, activation='relu'),
    Dense(num_classes, activation='softmax')
])

fusion_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [40]:
fusion_model.fit(stacked_probabilities_train, y_train_encoded, validation_data=(stacked_probabilities_test, y_val_encoded), epochs=10, batch_size=32)


Epoch 1/10
11/11 [==============================] - 1s 25ms/step - loss: 1.3311 - accuracy: 0.3172 - val_loss: 1.2563 - val_accuracy: 0.3902
Epoch 2/10
11/11 [==============================] - 0s 6ms/step - loss: 1.2082 - accuracy: 0.5257 - val_loss: 1.1517 - val_accuracy: 0.6585
Epoch 3/10
11/11 [==============================] - 0s 6ms/step - loss: 1.1071 - accuracy: 0.6858 - val_loss: 1.0481 - val_accuracy: 0.7439
Epoch 4/10
11/11 [==============================] - 0s 6ms/step - loss: 0.9942 - accuracy: 0.7764 - val_loss: 0.9417 - val_accuracy: 0.7805
Epoch 5/10
11/11 [==============================] - 0s 6ms/step - loss: 0.8584 - accuracy: 0.8248 - val_loss: 0.8380 - val_accuracy: 0.7805
Epoch 6/10
11/11 [==============================] - 0s 7ms/step - loss: 0.7192 - accuracy: 0.8640 - val_loss: 0.7382 - val_accuracy: 0.7927
Epoch 7/10
11/11 [==============================] - 0s 6ms/step - loss: 0.6143 - accuracy: 0.8731 - val_loss: 0.6464 - val_accuracy: 0.8049
Epoch 8/10
11/11 [=

In [41]:
val_loss, val_accuracy = fusion_model.evaluate(stacked_probabilities_test, y_val_encoded)
print(f"Validation Accuracy: {val_accuracy:.4f}")

3/3 [==============================] - 0s 3ms/step - loss: 0.4913 - accuracy: 0.8659
Validation Accuracy: 0.8659


In [60]:
num_classes = 4  # Assuming y_train is one-hot encoded

fusion_model = Sequential([
    Dense(64, input_shape=(stacked_probabilities_train.shape[1],), activation='relu'),
    Dropout(0.5),
    Dense(32, activation='relu'),
    Dropout(0.3),
    Dense(num_classes, activation='softmax')
])

fusion_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [61]:
fusion_model.fit(stacked_probabilities_train, y_train_encoded, validation_data=(stacked_probabilities_test, y_val_encoded), epochs=10, batch_size=32)

Epoch 1/10
11/11 [==============================] - 1s 30ms/step - loss: 1.2804 - accuracy: 0.4502 - val_loss: 1.2606 - val_accuracy: 0.6585
Epoch 2/10
11/11 [==============================] - 0s 6ms/step - loss: 1.2032 - accuracy: 0.5982 - val_loss: 1.1628 - val_accuracy: 0.7805
Epoch 3/10
11/11 [==============================] - 0s 6ms/step - loss: 1.0850 - accuracy: 0.7553 - val_loss: 1.0573 - val_accuracy: 0.8049
Epoch 4/10
11/11 [==============================] - 0s 6ms/step - loss: 0.9527 - accuracy: 0.7976 - val_loss: 0.9446 - val_accuracy: 0.7927
Epoch 5/10
11/11 [==============================] - 0s 5ms/step - loss: 0.8662 - accuracy: 0.8338 - val_loss: 0.8319 - val_accuracy: 0.8171
Epoch 6/10
11/11 [==============================] - 0s 6ms/step - loss: 0.7168 - accuracy: 0.8489 - val_loss: 0.7301 - val_accuracy: 0.8293
Epoch 7/10
11/11 [==============================] - 0s 6ms/step - loss: 0.6021 - accuracy: 0.8761 - val_loss: 0.6423 - val_accuracy: 0.8171
Epoch 8/10
11/11 [=

In [62]:
val_loss, val_accuracy = fusion_model.evaluate(stacked_probabilities_test, y_val_encoded)
print(f"Validation Accuracy: {val_accuracy:.4f}")

3/3 [==============================] - 0s 3ms/step - loss: 0.5037 - accuracy: 0.8659
Validation Accuracy: 0.8659


In [75]:
y_pred_test_nn = fusion_model.predict(stacked_probabilities_test)
predicted_classes = np.argmax(y_pred_test_nn, axis=1)


3/3 [==============================] - 0s 2ms/step


In [73]:
y_encoded_test

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3])

In [76]:
print(classification_report(y_encoded_test, predicted_classes, target_names=label_encoder.classes_))


                                                      precision    recall  f1-score   support

                                   Atopic Dermatitis       0.82      1.00      0.90        32
          Lupus and other Connective Tissue diseases       0.94      0.88      0.91        17
Scabies Lyme Diease and other Infestations and Bites       0.86      0.72      0.78        25
        Seborrheic Keratoses and other Benign Tumors       1.00      0.75      0.86         8

                                            accuracy                           0.87        82
                                           macro avg       0.90      0.84      0.86        82
                                        weighted avg       0.87      0.87      0.86        82



In [54]:
num_classes = 4  # Assuming y_train is one-hot encoded

fusion_model = Sequential([
    Dense(64, input_shape=(stacked_probabilities_train.shape[1],), activation='relu',kernel_regularizer=l2(0.01)),
    Dropout(0.5),
    Dense(32, activation='relu'),
    Dropout(0.3),
    Dense(num_classes, activation='softmax')
])

fusion_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [55]:
fusion_model.fit(stacked_probabilities_train, y_train_encoded, validation_data=(stacked_probabilities_test, y_val_encoded), epochs=10, batch_size=32)

Epoch 1/10
11/11 [==============================] - 2s 28ms/step - loss: 1.4608 - accuracy: 0.3474 - val_loss: 1.3767 - val_accuracy: 0.5976
Epoch 2/10
11/11 [==============================] - 0s 6ms/step - loss: 1.3295 - accuracy: 0.5740 - val_loss: 1.2609 - val_accuracy: 0.8049
Epoch 3/10
11/11 [==============================] - 0s 7ms/step - loss: 1.2011 - accuracy: 0.7372 - val_loss: 1.1421 - val_accuracy: 0.7927
Epoch 4/10
11/11 [==============================] - 0s 6ms/step - loss: 1.0555 - accuracy: 0.7885 - val_loss: 1.0220 - val_accuracy: 0.7927
Epoch 5/10
11/11 [==============================] - 0s 6ms/step - loss: 0.9378 - accuracy: 0.8308 - val_loss: 0.9104 - val_accuracy: 0.7927
Epoch 6/10
11/11 [==============================] - 0s 7ms/step - loss: 0.8285 - accuracy: 0.8399 - val_loss: 0.8121 - val_accuracy: 0.7927
Epoch 7/10
11/11 [==============================] - 0s 16ms/step - loss: 0.7330 - accuracy: 0.8489 - val_loss: 0.7352 - val_accuracy: 0.7927
Epoch 8/10
11/11 [

In [56]:
val_loss, val_accuracy = fusion_model.evaluate(stacked_probabilities_test, y_val_encoded)
print(f"Validation Accuracy: {val_accuracy:.4f}")

3/3 [==============================] - 0s 3ms/step - loss: 0.6166 - accuracy: 0.8293
Validation Accuracy: 0.8293
